In [303]:
from bs4 import BeautifulSoup
from html.parser import HTMLParser
import requests
import json
import re
global str

In [384]:
def extract_book_info(book_id):
    book_url = "https://www.bookvoed.ru/book?id=" + book_id
    book_html = requests.get(book_url).text
    
    soup = BeautifulSoup(book_html, 'html.parser')
    
    #decription
    desc = soup.find('div', class_='lw')
    if desc is not None:
        desc_str = re.sub('\n', ' ', desc.text)
        desc_str = re.sub(' далее Читать Свернуть ', ' ', desc_str)
    else:
        desc_str = ''
    
    #age
    if soup.find('div', class_='qv nM') is not None:
        age = '12+'
    elif soup.find('div', class_='rv nM') is not None:
        age = '16+'
    elif soup.find('div', class_='sv nM') is not None:
        age = '18+'
    else:
        age = '0+'
        
    #bookmark
    bookmark_str = soup.find('a', class_='Ke Le ff').text.strip()
    if not bookmark_str:
        bookmark_int = 0
    else:
        bookmark_int = int(bookmark_str)

    #likes
    likes_str = soup.find('a', class_='Ke Me ').text.strip()
    if not likes_str:
        likes_int = 0
    else:
        likes_int = int(likes_str)

    #dislikes
    dislikes_str = soup.find('a', class_='Ke Oe ').text.strip()
    if not dislikes_str:
        dislikes_int = 0
    else:
        dislikes_int = int(dislikes_str)

    #rating
    if likes_int == dislikes_int:
        rating = 0
    else:
        rating = likes_int/(likes_int + dislikes_int)*100
    
    #цена 
    if soup.find('div', class_="S2") is not None:
        price = soup.find('div', class_="S2").text.strip()
    else:
        price = 'for free'
    book_info = {
    "ID": book_id,
    "Название": soup.find('h1', itemprop='name').text,
    "Обложка": soup.find('img')['src'],
    "Возраст": age,
    "Описание": desc_str,
    "Рейтинг": rating,  
    "Понравилось": likes_int,
    "В закладки": bookmark_int,
    "Не понравилось": dislikes_int,
    "Цена": price,
}
        
    table = soup.find('table', class_='tw')
    rows = table.find_all('tr')
    data = [list(map(lambda x: x.text, row.find_all('td'))) for row in rows]
    for i in data:
        book_info.update({i[0] : i[1]})
    return book_info

In [385]:
book_info = extract_book_info("6832803")
book_info

{'ID': '6832803',
 'Название': 'Каникулы в Простофилино  Донцова Дарья Аркадьевна',
 'Обложка': '/files/1836/34/13/87/1.jpeg',
 'Возраст': '16+',
 'Описание': ' Я, Виола Тараканова, решила помочь Милке Каркиной: по дому подруги бродит… чучело крокодила, тотем кровожадного африканского племени. Ходит эта мумия и угрожает уничтожить всю семью! Глупости, такого не бывает! Вон его, на помойку! Но что это? Вот же он, опять у порога стоит… И родители Милки вдруг одновременно умирают. Неужели причина именно в проклятии мерзкой крокодильей мумии? Нет, я не верю в чертовщину. Чучело немедленно в огонь! А теперь пора взяться за дела людские… ',
 'Рейтинг': 85.71428571428571,
 'Понравилось': 12,
 'В закладки': 12,
 'Не понравилось': 2,
 'Цена': '107 руб.',
 'Серия:': 'Иронический детектив Д. Донцовой (нов. суперэкон.)',
 'Издательство:': 'Эксмо',
 'Год:': '2017',
 'Страниц:': '384',
 'Переплёт:': 'мягкий',
 'ISBN:': '978-5-699-90631-4',
 'Размеры:': '10,20 см x 16,50 см',
 'Формат:': '165.00mm x 

In [386]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep

In [388]:
author_url = "https://www.bookvoed.ru/author/books?id=author_id"
books_list_ind = []
with webdriver.Chrome() as driver:
    id = ['29369','29442', '102994', '26149', '35952']
    for i in id:
        url_current = re.sub('author_id', i, author_url)
        
        driver.get(url_current)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        match=False
        while(match==False):
            lastCount = lenOfPage
            sleep(2)
            lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount==lenOfPage:
                match=True
                
        items = driver.find_elements_by_class_name('gf')
        for item in items:
            classes = ''.join(item.get_attribute('data-book').split())
            books_list_ind.append(classes)
print(len(books_list_ind))
print(type(books_list_ind[0]))
print(books_list_ind[0])


1420
<class 'str'>
9340993


In [389]:
import pandas as pd

In [390]:
books_info_list = [extract_book_info(x) for x in books_list_ind]
print(len(books_info_list))

1420


In [409]:


df = pd.DataFrame(books_info_list)
print(df)

           ID              ISBN:                      Автор:  \
0     9340993  978-5-04-100203-9    Донцова Дарья Аркадьевна   
1     9340995  978-5-04-099550-9    Донцова Дарья Аркадьевна   
2     9689115  978-5-04-100629-7    Донцова Дарья Аркадьевна   
3     8645468  978-5-04-096033-0    Донцова Дарья Аркадьевна   
4     8835701  978-5-04-097265-4    Донцова Дарья Аркадьевна   
5     8867666  978-5-04-098472-5    Донцова Дарья Аркадьевна   
6     9409235  978-5-04-100207-7    Донцова Дарья Аркадьевна   
7     8505180  978-5-04-093340-2    Донцова Дарья Аркадьевна   
8     7493745  978-5-699-99396-3    Донцова Дарья Аркадьевна   
9     8626310  978-5-04-094631-0    Донцова Дарья Аркадьевна   
10    7891402  978-5-04-091460-9    Донцова Дарья Аркадьевна   
11    8626305  978-5-04-095389-9    Донцова Дарья Аркадьевна   
12    8550136  978-5-04-094170-4    Донцова Дарья Аркадьевна   
13    9340994  978-5-04-099140-2    Донцова Дарья Аркадьевна   
14    9293590  978-5-04-098723-8    Донц

In [410]:
df = pd.DataFrame(books_info_list)
#df.sort_values(by=['ID'], inplace=True)

with open('hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)